<a href="https://colab.research.google.com/github/yuce-8/alpha/blob/main/YUCE_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YUCE-8.tech API documentation

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install requests

import json
import requests

## Connect to YUCE-8
this is subject to change in the next versions

In [ ]:
class GCF():
  def __init__(self):
    self.url = '<put in here>'
    print('YUCE-8 ready @ ', self.url)

  def push(self, json_obj, return_result=False):
    r = requests.post(self.url, json=json_obj)
    if return_result:
      return r

yuce_8 = GCF()

## Test connection
the test will return the current backend version

In [3]:
def get_current_backend_version():
   req = yuce_8.push({'action': 'get_version'}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']


print(get_current_backend_version())

v0.1.1 alpha


## List all available forecasts

In [4]:
def get_available_forecasts():
   req = yuce_8.push({'action': 'get_available_forecasts'}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']

print(get_available_forecasts())

[{'BTC-USD': ['1d']}]


## Get latest forecast

In [6]:
def get_latest_forecast(symbol, interval):
   req = yuce_8.push(
      {
        'action': 'get_forecast',
        'symbol': symbol,
        'interval': interval
      }, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']['yuce_8']
   else:
     return ';-( ' + req.json()['msg']

# get the forecast:
f_0 = get_latest_forecast('BTC-USD', '1d')

# print out the content:
print('when was the forecast created?', f_0['forecast_created_at'])
print('what is the last quote that was used to forecast?', f_0['last_quote'], ' @ ', f_0['last_date'])
print('what is the name of the magic model that was used to forecast?', f_0['model_name'], 'last trained @', f_0['training_date'])
print('what is the forecasted top range?', f_0['quotes_forecast_top'])
print('what is the forecasted bottom range?', f_0['quotes_forecast_bottom'])

# display the forecast timestep per timestep:
print('timestep', '0 = ', f_0['last_quote'], ' @ ', f_0['last_date'])
quote_0 = f_0['last_quote']
top_range = f_0['quotes_forecast_top']
bottom_range = f_0['quotes_forecast_bottom']

for i in range(0, len(top_range)):
  forecast_top_at_i = top_range[i]
  forecast_bottom_at_i = bottom_range[i] 
  
  # see if it is a bearish or bullish forecast
  bearish = forecast_top_at_i <= quote_0

  print('timestep', i+1, 'should be w/in', ']' if bearish else '[', forecast_bottom_at_i, ', ', forecast_top_at_i, ']' if bearish else '[')


when was the forecast created? 2021-02-02 11:48:48.986973+01:00
what is the last quote that was used to forecast? 33157.5  @  2021-02-01 22:00:00
what is the name of the magic model that was used to forecast? Model 2 last trained @ 2021-02-02 10:40:54
what is the forecasted top range? [33240.393749999996, 33240.393749999996, 33240.393749999996, 33240.393749999996, 33240.393749999996, 33240.393749999996, 33240.393749999996, 33240.393749999996, 33240.393749999996, 33240.393749999996, 33240.393749999996, 33240.393749999996, 33240.393749999996]
what is the forecasted bottom range? [33157.5, 33157.5, 33157.5, 33157.5, 33157.5, 33157.5, 33157.5, 33157.5, 33157.5, 33157.5, 33157.5, 33157.5, 33157.5]
timestep 0 =  33157.5  @  2021-02-01 22:00:00
timestep 1 should be w/in [ 33157.5 ,  33240.393749999996 [
timestep 2 should be w/in [ 33157.5 ,  33240.393749999996 [
timestep 3 should be w/in [ 33157.5 ,  33240.393749999996 [
timestep 4 should be w/in [ 33157.5 ,  33240.393749999996 [
timestep 5 s

## Backtesting the results

to be done